In [ ]:

from pathlib import Path
from netCDF4 import Dataset,chartostring
import numpy as np
import os
import datetime 
import pandas as pd
import matplotlib.pyplot as plt


  


In [ ]:
#you need to set these!
WHATPRIOR='na_sword_v16_SOS_priors.nc'
WHATAGENCY='USGS'
WHATFLPE='sic4dvar'
QVAR='Q_da'

In [ ]:
#this is where this run currentlly lives
RUN=Path('/nas/cee-water/cjgleason/SWOT_Q_UMASS')
SONE=RUN.joinpath('mnt').joinpath('flpe')
TSfiles=RUN.joinpath('mnt').joinpath('input').joinpath('swot')
YOURFLPE=SONE.joinpath(WHATFLPE)
if WHATFLPE == 'sic4dvar':
    YOURFLPE=YOURFLPE.joinpath('reaches')
    


In [ ]:
#get all the directories/files lined up
G=RUN.joinpath(WHATPRIOR)
G=Dataset(G)
Grid=G[WHATAGENCY][WHATAGENCY+'_reach_id'][:].filled(np.nan)
#overlap in FLPE files
yourflpefiles=os.listdir(YOURFLPE)
FLPEr=[]
for filen in yourflpefiles:
    D=Dataset(YOURFLPE.joinpath(filen))
    if any(~np.isnan(D[QVAR][:].filled(np.nan))):
        FLPEr.append(int(filen[0:11]))

OLR=list(set(FLPEr).intersection(set(Grid)))

In [ ]:
#grab all the time series data
FLPEQ=[]
ST=[]
GT=[]
GQ=[]
Reach=[]
for REACH in OLR:
    Reach.append(REACH)
    TSfile=Dataset(TSfiles.joinpath(str(REACH)+'_SWOT.nc'))
    ttt=chartostring(TSfile['reach/time_str'][:])
    reachT=[]
    for TIMZ in ttt:
        if TIMZ=='no_data':
            reachT.append(np.nan)
        else:
            reachT.append( datetime.datetime.strptime(TIMZ[0:10],'%Y-%m-%d').toordinal())
        
    ST.append(reachT)
    FLPEf=Dataset(YOURFLPE.joinpath(str(REACH)+'_'+WHATFLPE+'.nc'))
    FLPEQ.append(FLPEf[QVAR][:].filled(np.nan))
    Gindex=np.where(G[WHATAGENCY][WHATAGENCY+'_reach_id'][:].filled(np.nan)==REACH)[0][0]
    GQ.append(G[WHATAGENCY+'/'+WHATAGENCY+'_q'][Gindex].filled(np.nan))
    GT.append(G[WHATAGENCY+'/'+WHATAGENCY+'_qt'][Gindex].filled(np.nan))
        
 


In [ ]:
COMPdf=pd.DataFrame()
COMPdf['Reach']=pd.Series(Reach)
COMPdf['ST']=pd.Series(ST)
COMPdf['GT']=pd.Series(GT)
COMPdf['GQ']=pd.Series(GQ)
COMPdf['FLPEQ']=pd.Series(FLPEQ)


In [ ]:
#clean up and index time series
indexedgq=[]
st=[]
soneq=[]
rid=[]
for r in range(len(COMPdf.Reach)):
    rdf=COMPdf.loc[r]
    rid.append(rdf.Reach)
    STfilter=~np.isnan(rdf.ST)
    goodst=np.array(rdf.ST)[STfilter]
    goodflpeq=rdf.FLPEQ[STfilter]
    gqindex=[]
    for time in goodst:
        tindex=np.where(rdf.GT[~np.isnan(rdf.GT)].astype(int)==int(time))[0]
        if np.size(tindex)>0:
            gqindex.append(tindex[0])
        else:
            gqindex.append(np.nan)
    
    Ggqindex=~np.isnan(gqindex)
    indexedgq.append(rdf.GQ[~np.isnan(rdf.GT)][np.array(gqindex).astype(int)[Ggqindex]])
    st.append(goodst[Ggqindex])
    soneq.append(goodflpeq[Ggqindex])




In [ ]:
fCOMPdf=pd.DataFrame()
fCOMPdf['Reach']=pd.Series(rid)
fCOMPdf['ST']=pd.Series(st)
fCOMPdf['GQ']=pd.Series(indexedgq)
fCOMPdf['FLPEQ']=pd.Series(soneq)

In [ ]:
#run stats
NSEo=[]
Rsqo=[]
KGEo=[]
RMSEo=[]
nRMSEo=[]
nBIASo=[]
rRMSEo=[]
no=[]
reach=[]
for re in range(len(fCOMPdf['Reach'])):
    reach.append(fCOMPdf.loc[re].Reach)
    GQ=fCOMPdf.loc[re]['GQ'][~np.isnan(fCOMPdf.loc[re]['FLPEQ'])]
    SQ=fCOMPdf.loc[re]['FLPEQ'][~np.isnan(fCOMPdf.loc[re]['FLPEQ'])] 
    
   

    # NSE
    top=np.sum((GQ-SQ)**2)
    bottom=np.sum((GQ-np.mean(GQ))**2)
    NSE=1-(top/bottom)
    NSEo.append(NSE)
    #Rsq
    r=np.corrcoef( SQ,GQ)
    r=r[0,1]
    Rsq=r**2
    Rsqo.append(Rsq)
    #KGE
    KGE=1-np.sqrt((r-1)**2 + ((np.std( SQ)/np.std(GQ))-1)**2 +((np.mean(SQ)/np.mean(GQ))-1)**2)  
    KGEo.append(KGE)
    #n
    n=len(GQ)
    no.append(n)
    #RMSE
    RMSE=np.sqrt((np.sum( (SQ - GQ)**2))/n)
    RMSEo.append(RMSE)                
    #nRMSE
    NRMSE=RMSE/np.mean(GQ)
    nRMSEo.append(NRMSE)
    #nBIASo
    BIAS= np.sum(SQ - GQ)/len(GQ)
    nBIAS=BIAS/np.mean(GQ)
    nBIASo.append(nBIAS)
    #rRMSE
    rRMSEo.append(np.sqrt(NRMSE**2-nBIAS**2))
    
STATS=pd.DataFrame()
STATS['reach_id']=pd.Series(reach)
STATS['r2']=pd.Series(Rsqo)
STATS['NSE']=pd.Series(NSEo)
STATS['KGE']=pd.Series(KGEo)
STATS['RMSE']=pd.Series(RMSEo)
STATS['nRMSE']=pd.Series(nRMSEo)
STATS['nBIAS']=pd.Series(nBIASo)
STATS['rRMSE']=pd.Series(rRMSEo)
STATS['n']=pd.Series(no)
STATS

In [ ]:
#here's a friendly little cell to index the table obove
np.max(STATS.n)
STATS.loc[STATS.n==9]

In [ ]:
#Use you index here! 
Pindex=85
T=fCOMPdf.loc[Pindex]['ST']
pt=[]
for t in T:
    pt.append(datetime.date.fromordinal(int(t)))
T=pt
S=fCOMPdf.loc[Pindex]['FLPEQ']
G=fCOMPdf.loc[Pindex]['GQ']
plt.plot(T, S,linestyle='--', marker='o', label=WHATFLPE)
plt.plot(T,G,linestyle='--', marker='o', label='gage')
plt.ylabel("Q (m^3)")
plt.title('Reach ID '+ str(fCOMPdf.loc[Pindex].Reach))
plt.legend()